# Visualise latent space

The aim of this notebook is to replicate a figure from the lecture notes of _Cambridge Engineering-Deep Learning for Computer Vision_. The task is binary classification with a 3 node hidden layer. This hidden layer is visualised in 3D space, which showed that the action of learning _pushed_ the weights into corners of the cube.

In [12]:
# Relative import hack
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))

# Setup logging
import pickle
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib.animation as ani
from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from digit_classifier import base, DigitClassifier, DEVICE, Metrics, DigitClassifierConfig, train_test_datasets, train_val_dataset_split
import logging

sns.set_theme()

In [13]:
base() # Set seeds

In [14]:
# Model parameters
config = DigitClassifierConfig(
	sizes=[784, 10, 3, 10],
	learning_rate=1,
  device=DEVICE,
  loss = nn.MSELoss(reduction='mean'),
  mini_batch = 20,
)

model_dir = Path("../../resources/model/latent_space/")
metrics_dir = model_dir / 'metrics.pkl'

In [15]:
# Extract the labels of the training set
train_set = train_val_dataset_split(train_test_datasets()[0])[0]
labels = [train_set[i][1] for i in range(len(train_set))]
labels = np.array(labels)
labels.shape

[digit_classifier] [INFO] Train size: 60000
[digit_classifier] [INFO] Test size: 10000
[digit_classifier] [INFO] Train set: 50000
[digit_classifier] [INFO] Valid set: 10000


(50000,)

In [11]:
INSPECT_LAYER = 1  # Which layer to inspect

# Variable to store the latent space output and classifier label
latent_label = np.zeros(
    (1,  # Epoch: increment this axis each epoch
     1,  # Training example:
     config.sizes[1]  # Store latents
     + 1))  # +1 for the classifier label
latent_label

array([[[0., 0., 0., 0.]]])

In [29]:
latent_label = []
latent_dataloader = torch.utils.data.DataLoader(train_set, batch_size=1000, shuffle=False,num_workers=2, drop_last=False)

@torch.no_grad()
def func(dc: DigitClassifier, **func_kwargs):
  """Keep track of the latent space output

  Args:
    dc: DigitClassifier
    ll: np array to store the outputs

  After each batch/epoch, evaluate on the training set (with no grad)
  for each data point
  - run a forward pass
  - keep track of the latent space, and the output
  """
  ll: list = func_kwargs['ll']

  dc.eval()
  # Increment ll axis 0
  ll.append([])
  # ll = np.append(ll, np.zeros((1, 1, config.sizes[1] + 1)), axis=0)
  for x, y in latent_dataloader:
    for layer_idx, layer in enumerate(dc.linears):
      x = layer(x)
      x = dc.act_fn(x)
      if layer_idx == INSPECT_LAYER:
        break
    latent_space = x.detach().numpy()
    # latent_space = np.append(latent_space, y.cpu().numpy()[:,None], axis=1)
    ll[-1] += latent_space.tolist()

## Train Model

In [30]:
model = DigitClassifier(config)
model

[digit_classifier] [INFO] Train size: 60000
[digit_classifier] [INFO] Test size: 10000
[digit_classifier] [INFO] Train set: 50000
[digit_classifier] [INFO] Valid set: 10000


DigitClassifier(
  (act_fn): Sigmoid()
  (linears): ModuleList(
    (0): Linear(in_features=784, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=3, bias=True)
    (2): Linear(in_features=3, out_features=10, bias=True)
  )
  (loss_module): MSELoss()
)

In [31]:
epochs = 2
metrics = Metrics()
model.train_loop(num_epochs=epochs, metrics=metrics, func=func, ll=latent_label)

[digit_classifier] [INFO] Epoch: 0: 1149 / 10000
[digit_classifier] [INFO] Epoch: 1: 2422 / 10000


In [33]:
# Save model
model.save_model(model_dir)

(WindowsPath('../../resources/model/latent_space/model.tar'),
 WindowsPath('../../resources/model/latent_space/config.pkl'))

In [34]:
latent_label = np.array(latent_label)
latent_label.shape

(2, 50000, 3)

In [35]:
# Convert to numpy array and save
f = model_dir / f'latents-{epochs}.npy'
np.save(f, latent_label)
f = model_dir / f'labels.npy'
np.save(f, labels)